In [5]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import ftplib
from io import BytesIO
from PIL import Image, ImageDraw
import os
import ftplib
import pandas as pd
from glob import glob
import imageio.v3 as iio


In [6]:
#defining start and end time
start = '2023-08-01'
end = '2023-09-01'

 

In [7]:
#loading data
d = xr.open_dataset('/Users/landergreenway/Downloads/School/JuniorYr/EV333_AtmosphericDynamics/python/FinalProject/atmorivers/ERA5_SouthAmerica_2023_hourly_tcwv.nc')

#subsetting data to average over i
d = d.sel(time = slice(start, end))
d = d['tcwv']

#defining number of frames in given time period
num = len(d['time'])
print(num)
print(d)

768
<xarray.DataArray 'tcwv' (time: 768, latitude: 181, longitude: 341)>
[47401728 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 -140.0 -139.8 -139.5 ... -55.5 -55.25 -55.0
  * latitude   (latitude) float32 -15.0 -15.25 -15.5 ... -59.5 -59.75 -60.0
  * time       (time) datetime64[ns] 2023-08-01 ... 2023-09-01T23:00:00
Attributes:
    units:          kg m**-2
    long_name:      Total column vertically-integrated water vapour
    standard_name:  lwe_thickness_of_atmosphere_mass_content_of_water_vapor


In [16]:
#data vis and projection
cmap = cmocean.cm.thermal
lev = np.arange(0, 20, 0.5)
proj = ccrs.PlateCarree(central_longitude=-77)

#creating loop
for i in range(450, 455):

    fig = plt.figure(figsize = (9, 4.5), dpi = 300)
    ax = plt.axes(projection = proj)
    
    # filled contour map of IVT transfer
    d[i].plot.contourf(
        x = 'longitude',
        y = 'latitude',
        ax=ax,
        transform=ccrs.PlateCarree(),
        levels=lev,
        extend='both',
        colors=cmap,
        add_colorbar=True,
        cbar_kwargs = {"label":"Water Vapor Transport (kg/m^2)"})
    # add coastlines
    ax.coastlines(
       resolution='50m')  #Currently can be one of “110m”, “50m”, and “10m”.

# add coastlines masking the land in lightgray
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '110m', edgecolor='k', facecolor='lightgray'))

# add grid lines
    gl = ax.gridlines(crs=ccrs.PlateCarree(),
                  draw_labels=True,
                  linewidth=1,
                  color='gray',
                  alpha=0.5,
                  linestyle='--')

#extract year, month, day, and hour from the time series using splits, time format YYY-MM-DD-T00:00:00.000000000
    times = d['time'][i].values.astype('datetime64[s]')
    yr = str(np.datetime64(times, 'Y').astype(str))
    mon = str(np.datetime64(times, 'M').astype(str)).split('-')[1]
    day = str(np.datetime64(times, 'D').astype(str)).split('-')[2]
    hr = str(np.datetime64(times, 'h').astype(str)).split('T')[1].split(':')[0]

# add title
    ax.set_title("IVT test on " + mon + ', ' + yr + ' at ' + hr + ':00')
#constructing filename
    name = f'RiversTestIVT_{yr}-{mon}-{day}-{hr}.png'

    #saveFolder = '/Users/landergreenway/Downloads/School/JuniorYr/EV333_AtmosphericDynamics/python/FinalProject/atmorivers/AtmoRiversFrames'
    
    
# save figure 
    fig.savefig('AtmoRiversIVTFrames/' + name, facecolor = 'white', transparent = False, bbox_inches ='tight')
   # fig.savefig(name, facecolor = 'white', transparent = False, bbox_inches ='tight')

    plt.close(fig)
    #print(name)
     
    

In [17]:
#Making GIFs

#directory = '/AtmoriversIVTFrames/'
#filenames = 'RiversTestIVT*.png'


files = sorted(glob('AtmoRiversIVTFrames/RiversTestIVT*.png'))
files



['AtmoRiversIVTFrames/RiversTestIVT_2023-08-19-18.png',
 'AtmoRiversIVTFrames/RiversTestIVT_2023-08-19-19.png',
 'AtmoRiversIVTFrames/RiversTestIVT_2023-08-19-20.png',
 'AtmoRiversIVTFrames/RiversTestIVT_2023-08-19-21.png',
 'AtmoRiversIVTFrames/RiversTestIVT_2023-08-19-22.png']

In [21]:

#creating array
images = [ ] 

#setting loop 
for filename in files:
        images.append(iio.imread(filename))

iio.imwrite('RiversTestIVT.gif', images, duration = 500, loop = 0)


In [ ]:
def pull_plot(region, start_date, end_date):
    
    
    """ This function takes the input region, start date, and end date, obtains the satellite 
        data from the ssec.wisc.edu ftp server and generates an output .gif saved to the local 
        machine. It does not store files temporarily on local machine. 
    
        region: string, input region from bulleted list below
        start_date: string of date to start loop featuring day, month, and year in any order using either / or - 
        end_date: string of date to start loop featuring day, month, and year in any order using either / or -     
    
    """
    
    
    
    date_range = pd.date_range(start = start_date, end = end_date, freq = 'H')                 ## establish date range given inputs 
    months = pd.unique([i.strftime('%Y%m') for i in date_range])                               ## create unique Year/Month array to loop through ftp directory based on years and months
                                                                                               ## this is somewhat of a saftey net if event desires straddles months or years
    
    ftp = ftplib.FTP('ftp.ssec.wisc.edu', 'anonymous')                                         ## log into the FTP server

    images = []                                                                                ## create empty array to store files obtained from looping through ftp directories

    for yearmon in months:                                                                     ## loop through unique years/months desired
            
        try:
    '''
            ftp.cwd(f"/pub/mtpw2/images/tpw_nrl_colors/{region}/{yearmon}")                    ## cd into correct directory based on region and year/month desired
        
            for pic in sorted(ftp.nlst()):                                                     ## ftp.nlst lists all the files in our ftp directory accessed above, we are looping over each image in this list

                r = BytesIO()  sub = pd.to_datetime(str(pic)[4:15], format = '%Y%m%d.%H')                     ## ensure the image selected is within date range

                if sub >= date_range[0] and sub <= date_range[-1]:

                    print(f'{sub} is downloading')

                    ftp.retrbinary(f'RETR {pic}', r.write)                                     ## retrive the image from the list and store it in our established byte buffer

                    images.append(r)                                                           ## append the byte buffer for later plotting
        
        except:

            print('Invalid region or date range inputted')
            
        
    img, *imgs = [Image.open(f) for f in images]                                               ## formally open each temporarily stored byte buffer in images list
    
    img.save(f'IWV_{region}_{str(date_range[0])[0:10]}_{str(date_range[-1])[0:10]}.gif',       ## utilize image gif plotter that accepts output name (first row here is the saved output name based on region and dates)
             format='GIF', append_images=imgs, save_all=True, duration=200, loop=3)            ## and append_images = our list of opened images that are in byte form
                                                                                               ## gif features can be altered here such as # of loops and speed     

    ftp.quit()                                                                                 ## make sure to close our connection!! 
    '''
